In [1]:
import MDAnalysis as mda
import numpy as np
import matplotlib.pyplot as plt
import sys,os
import math

from tqdm.autonotebook import tqdm as tq
import torch
from gridData import OpenDX
from MDAnalysis.analysis import align
from MDAnalysis.analysis.rms import rmsd
#from MDAnalysis import transformations as trans
from MDAnalysis.transformations import translate
from scipy.interpolate import griddata
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
import psutil

In [2]:
###Again, the following two lines can be selectively ignored
p = psutil.Process(os.getpid())
p.cpu_affinity(list(range(16)))
###Select the GPU ID used for calculation
device1 = torch.device("cuda:0")
def write_pdb(name, coordinate):
    pdb = open(name, 'w')
    counter = 1
    for i in coordinate:
        pdb.write('ATOM  %5d  OH2 PTH     1    %8.3f%8.3f%8.3f%6.2f%6.2f\n'%(counter,i[0],i[1],i[2],0.0,0.0))
    return

In [3]:
###read in aligned_copied_water_traj
u_aligned = mda.Universe('copied_pdbs/pdbs/expanded_0000.pdb', 
                         'copied_pdbs/1jwp_oh2_copied_aligned.dcd')

###Origin Traj is needed for dimensions

u = mda.Universe('trajectory/1jwp_noh.pdb', 
                 'trajectory/1jwp_noh.dcd')
# Read the dimension
dim_float = u.trajectory.ts.dimensions[0]

# Round up to the next integer
dim_rounded = math.ceil(dim_float)

# If the rounded dimension is odd, add 1 to make it even
if dim_rounded % 2 != 0:
    dim_rounded += 1

dim = dim_rounded

###Define Grid Box1
protein_box = np.array([dim, dim, dim]) 
# Define the resolution of the grid box
resolution = 1  # in Angstroms

# Determine the number of grid points along each axis
n_points = np.ceil(protein_box / resolution).astype(int)

# Determine the box dimensions of the grid box
grid_box = n_points * resolution

# Define the coordinates of the grid points
x_coords = np.linspace(-grid_box[0]/2, grid_box[0]/2, n_points[0])
y_coords = np.linspace(-grid_box[1]/2, grid_box[1]/2, n_points[1])
z_coords = np.linspace(-grid_box[2]/2, grid_box[2]/2, n_points[2])

# Create a 3D grid of the coordinates
xx, yy, zz = np.meshgrid(x_coords, y_coords, z_coords, indexing='ij')

grid_coords = np.stack((xx,yy,zz), axis=-1)
num_grids = np.prod(n_points)
grid_coords1 = grid_coords.reshape((num_grids), 3)

# Define the reolution of the grid box2
resolution2 = 2  # in Angstroms

# Determine the number of grid points along each axis
n_points2 = np.ceil(protein_box / resolution2).astype(int)

# Determine the box dimensions of the grid box
grid_box2 = n_points2 * resolution2

# Define the coordinates of the grid points
x_coords2 = np.linspace(-grid_box2[0]/2, grid_box2[0]/2, n_points2[0])
y_coords2 = np.linspace(-grid_box2[1]/2, grid_box2[1]/2, n_points2[1])
z_coords2 = np.linspace(-grid_box2[2]/2, grid_box2[2]/2, n_points2[2])

# Create a 3D grid of the coordinates
xx2, yy2, zz2 = np.meshgrid(x_coords2, y_coords2, z_coords2, indexing='ij')

grid_coords2 = np.stack((xx2,yy2,zz2), axis=-1)
num_grids2 = np.prod(n_points2)
grid_coords2 = grid_coords2.reshape((num_grids2), 3)

/usr/local/anaconda3/envs/RCs/lib/python3.8/site-packages/MDAnalysis/topology/PDBParser.py:328: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/usr/local/anaconda3/envs/RCs/lib/python3.8/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [4]:
###Generate corners of the high-resolution grid box for inbox water selection
X_coords, Y_coords, Z_coords = zip(*grid_coords1)
min_X, max_X = min(X_coords), max(X_coords)
min_Y, max_Y = min(Y_coords), max(Y_coords)
min_Z, max_Z = min(Z_coords), max(Z_coords)
ow_within_cube = u_aligned.select_atoms('name OH2 and prop x > {min_X} and prop x < {max_X} and prop y > {min_Y} and prop y < {max_Y} and prop z > {min_Z} and prop z < {max_Z}'.format(min_X = min_X, max_X = max_X, min_Y = min_Y, max_Y = max_Y, min_Z = min_Z, max_Z = max_Z), updating=True)
ow_within_cube_coords = [ow_within_cube.positions for ts in tq(u_aligned.trajectory)]

###Gaussian Convolution fxn
def Gaussian_convolution(e,d,r):
    g = ((2 * torch.pi * (e ** 2)) ** (-1 * d / 2)) * torch.exp(-1 * torch.square(r.to(device1)) / (2 * (e ** 2) ))
    return g



  0%|          | 0/3205 [00:00<?, ?it/s]

In [6]:
###Here, the split number is used for spliting the large grid box into multiple small grib boxes. Because the Grid box under 1 angstrom usually has a very high dimension, which will lead to heavy computational cost. Our suggestion is to separate into 2^n number of small grid boxes, since in our codes definition the grid box dimension is always even.
split_number = 43 
Grid_Coords_Split = np.split(grid_coords1, split_number)

Density_map = []
Density_map_2A = []
Delta = np.array([2., 2., 2.])
Origin = np.array([min_X, min_Y, min_Z])
#for i in tq(range(len(ow_within_cube_coords))):
for i in tq(range(len(u_aligned.trajectory))):
    ow_coords_i = ow_within_cube_coords[i]
    inside_waters_torch = torch.from_numpy(ow_coords_i).to(torch.float16).to(device1)
    Water_Density = np.array([0,0,0])
    for j in range(43):
        Density_new = 0
        Grid_Coordinates_torch = torch.from_numpy(Grid_Coords_Split[j]).to(torch.float16).to(device1)
        dist = torch.cdist(Grid_Coordinates_torch.float(), inside_waters_torch.float())
        Gaussian = Gaussian_convolution(1.7682, 3, dist)
        Grid_density = torch.sum(Gaussian, 1)
        Grid_density_np = Grid_density.cpu().numpy()
        del dist
        del Grid_Coordinates_torch
        del Gaussian
        del Grid_density
        Water_Density = np.concatenate((Water_Density, Grid_density_np), axis = 0)
    Water_Density = Water_Density[3:]
    del inside_waters_torch
    #interpolator = RegularGridInterpolator(grid_coords1, Water_Density)
    #density_map_2A = interpolator(grid_coords2)
    density_map_2A = griddata(points=grid_coords1, values=Water_Density, xi=grid_coords2, method='nearest')
    Density_map_2A.append(density_map_2A)
    Density_map.append(Water_Density)
    Grid_Density_rs = density_map_2A.reshape(43,43,43)
    #Water_Density_rs = Water_Density.reshape(92, 92, 92)
    dx = OpenDX.field('density')
    dx.add('positions', OpenDX.gridpositions(1, Grid_Density_rs.shape, Origin, Delta))
    dx.add('connections', OpenDX.gridconnections(2, Grid_Density_rs.shape))
    dx.add('data', OpenDX.array(3, Grid_Density_rs))
    dx.write('water_density_maps/{:04d}.dx'.format(i))
    

  0%|          | 0/3205 [00:00<?, ?it/s]

KeyboardInterrupt: 